In [2]:
import cv2
import dlib
import math

# Cargar detector de rostros y predictor de landmarks
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Función para calcular el ángulo de inclinación (roll)
def calculate_head_tilt(eye_left, eye_right):
    delta_y = eye_right[1] - eye_left[1]
    delta_x = eye_right[0] - eye_left[0]
    angle = math.degrees(math.atan2(delta_y, delta_x))
    return angle

# Inicializar captura de video
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        # Coordenadas de los ojos
        left_eye = (landmarks.part(36).x, landmarks.part(36).y)
        right_eye = (landmarks.part(45).x, landmarks.part(45).y)

        # Calcular el ángulo de inclinación de la cabeza
        head_tilt_angle = calculate_head_tilt(left_eye, right_eye)
        
        # Mostrar el ángulo en la pantalla
        cv2.putText(frame, f"Tilt: {head_tilt_angle:.2f}", (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Dibujar puntos en los ojos
        cv2.circle(frame, left_eye, 5, (0, 255, 0), -1)
        cv2.circle(frame, right_eye, 5, (0, 255, 0), -1)

    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'dlib'

In [2]:
import cv2

# Cargar el clasificador Haar para la detección de rostros
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_mcs_mouth.xml')

# Iniciar la captura de video
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detección de rostro
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        # Dibujar un rectángulo alrededor del rostro detectado
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        roi_gray = gray[y:y+h, x:x+w]  # Área de interés dentro del rostro
        roi_color = frame[y:y+h, x:x+w]
    
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        for (ex, ey, ew, eh) in eyes:
            # Dibujar un rectángulo alrededor de los ojos detectados
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

        mouth = mouth_cascade.detectMultiScale(roi_gray, 1.7, 11)
        
        for (mx, my, mw, mh) in mouth:
            # Si la boca está muy abierta (bostezo)
            if my > h/2:  # Filtar detecciones de boca que estén muy cerca de los ojos
                cv2.rectangle(roi_color, (mx, my), (mx+mw, my+mh), (0, 0, 255), 2)
                print("Bostezo detectado: posible fatiga")


    cv2.imshow('Rostro', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'


In [3]:
import cv2
import os

# Cargar los archivos XML de Haar Cascade para la detección de rostro, ojos y boca
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier('haarcascade_mcs_mouth.xml')

# Capturar video desde la webcam
cap = cv2.VideoCapture(0)

# Variables para controlar la detección de bostezos
bostezos_detectados = 0
bostezo_en_curso = False

while True:

    ret, frame = cap.read()
    
    if not ret:
        print("Error al capturar el frame")
        break

    # Convertir a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detección de rostro
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Definir la región de interés para los ojos y la boca
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Detección de ojos
        eyes = eye_cascade.detectMultiScale(roi_gray)

        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

        # Detección de boca
        mouth = mouth_cascade.detectMultiScale(roi_gray, 1.7, 11)

        for (mx, my, mw, mh) in mouth:
            # Filtar detecciones de boca que estén muy cerca de los ojos
            if my > h/2:  # Se verifica que la boca esté más abajo que la mitad del rostro
                cv2.rectangle(roi_color, (mx, my), (mx+mw, my+mh), (0, 0, 255), 2)
                print("Bostezo detectado: posible fatiga")


                # Verificar si la boca está abierta lo suficiente para considerar que es un bostezo
                area_boca = mx * my

                # Mostrar el tamaño del rectángulo de la boca
                size_text = f'Size: {area_boca}'
                cv2.putText(roi_color, size_text, (mx, my - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
  
                if area_boca > 20000:  # Ajusta este valor según tus pruebas
                    if not bostezo_en_curso:
                        bostezos_detectados += 1
                        bostezo_en_curso = True
                        print("Bostezo detectado: posible fatiga")
                else:
                    bostezo_en_curso = False

    # Check if display is available
    if os.environ.get('DISPLAY', '') == '':
        print('No display found. Running in headless mode.')
    else:
        cv2.imshow('Detección de Rostros, Ojos y Boca', frame)
        # Mostrar el frame con las detecciones


    # Presionar 'q' para salir del loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()


No display found. Running in headless mode.


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1367: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvWaitKey'


In [1]:
import cv2

# Cargar los archivos XML de Haar Cascade para la detección de rostro, ojos y boca
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier( 'haarcascade_mcs_mouth.xml')

# Capturar video desde la webcam
cap = cv2.VideoCapture(0)

# Variables para controlar la detección de bostezos
bostezos_detectados = 0
bostezo_en_curso = False

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Error al capturar el frame")
        break

    # Convertir a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detección de rostro
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Definir la región de interés para los ojos y la boca
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Detección de ojos
        eyes = eye_cascade.detectMultiScale(roi_gray)

        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

        # Detección de boca
        mouth = mouth_cascade.detectMultiScale(roi_gray, 1.7, 11)

        # Comprobar si hay detección de boca
        if len(mouth) > 0:
            for (mx, my, mw, mh) in mouth:
                # Filtrar detecciones de boca que estén muy cerca de los ojos
                if my > h / 2:  # Se verifica que la boca esté más abajo que la mitad del rostro
                    # Dibuja el rectángulo alrededor de la boca
                    cv2.rectangle(roi_color, (mx, my), (mx+mw, my+mh), (0, 0, 255), 2)

                    # Mostrar el tamaño del rectángulo de la boca
                    size_text = f'Tamaño: {mw}x{mh}'
                    cv2.putText(roi_color, size_text, (mx, my - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    # Calcular el área de la boca
                    area_boca = mw * mh

                    # Verificar si la boca está abierta lo suficiente para considerar que es un bostezo
                    if area_boca > 1000:  # Ajusta este valor según tus pruebas
                        if not bostezo_en_curso:
                            bostezos_detectados += 1
                            bostezo_en_curso = True
                            print("Bostezo detectado: posible fatiga")
                    else:
                        bostezo_en_curso = False
        else:
            # Reiniciar la detección si no hay boca
            bostezo_en_curso = False

    # Mostrar el frame con las detecciones
    cv2.imshow('Deteccion de Rostros, Ojos y Boca', frame)

    # Presionar 'q' para salir del loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()


Bostezo detectado: posible fatiga
Bostezo detectado: posible fatiga
Bostezo detectado: posible fatiga
Bostezo detectado: posible fatiga
Bostezo detectado: posible fatiga
Bostezo detectado: posible fatiga
Bostezo detectado: posible fatiga
Bostezo detectado: posible fatiga
Bostezo detectado: posible fatiga
Bostezo detectado: posible fatiga
Bostezo detectado: posible fatiga
Bostezo detectado: posible fatiga


In [6]:

# Parámetro de tiempo de cierre de los ojos (frames)
consecutive_frames = 0
CLOSED_EYE_THRESH = 30  # Cambia este valor según el framerate del video

for (x, y, w, h) in faces:
    roi_gray = gray[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    
    if len(eyes) == 0:  # Si no se detectan ojos, puede indicar que están cerrados
        consecutive_frames += 1
    else:
        consecutive_frames = 0  # Reiniciar contador si se detectan ojos abiertos
    
    if consecutive_frames >= CLOSED_EYE_THRESH:
        print("¡Advertencia de somnolencia!")
        # Puedes emitir un sonido o alerta aquí



In [ ]:
for (x, y, w, h) in faces:
    # Establece el centro del rostro
    face_center_x = x + w // 2
    face_center_y = y + h // 2

    # Establece el centro de la pantalla
    frame_center_x = frame.shape[1] // 2
    frame_center_y = frame.shape[0] // 2

    if abs(face_center_x - frame_center_x) > threshold or abs(face_center_y - frame_center_y) > threshold:
        print("Desviación de cabeza detectada: El conductor no está mirando al frente")


In [ ]:
mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_mcs_mouth.xml')

# Detección de boca abierta (bostezo)
for (x, y, w, h) in faces:
    roi_gray = gray[y:y+h, x:x+w]
    mouth = mouth_cascade.detectMultiScale(roi_gray, 1.7, 11)
    
    for (mx, my, mw, mh) in mouth:
        # Si la boca está muy abierta (bostezo)
        if my > h/2:  # Filtar detecciones de boca que estén muy cerca de los ojos
            cv2.rectangle(roi_color, (mx, my), (mx+mw, my+mh), (0, 0, 255), 2)
            print("Bostezo detectado: posible fatiga")